In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
##Langsmith tracking
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [5]:
from langchain_community.document_loaders import WebBaseLoader

In [ ]:
#READS THE ARTICLE FROM THE WEB
web_loader = WebBaseLoader("https://www.oreilly.com/radar/mcp-what-it-is-and-why-it-matters-part-3/")
web_loader

In [ ]:
# Load the web document
web_documents = web_loader.load()
web_loader

In [13]:
### Load data----> Docs ----> Divide into chunks ---->Vectors ---->Vector Embeddings ---> Store in VectorDB

from langchain.text_splitter import RecursiveCharacterTextSplitter
# Split the document into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splitted_documents = text_splitter.split_documents(web_documents)

In [14]:
splitted_documents

[Document(metadata={'source': 'https://www.oreilly.com/radar/mcp-what-it-is-and-why-it-matters-part-3/', 'title': '\nMCP: What It Is and Why It Matters—Part 3 – O’Reilly', 'description': '', 'language': 'en-US'}, page_content='MCP: What It Is and Why It Matters—Part 3 – O’Reilly          Skip to main content\nFor Enterprise\nFor Government\nFor Higher Ed\nFor Individuals\nFor Content Marketing\n\n\n\n\nFor Enterprise\nFor Government\nFor Higher Ed\nFor Individuals\nFor Content Marketing\nExplore Skills\n\nFeatures\nAll Features\nCourses\nCertifications\nInteractive Learning\nLive Events\nAnswers\nInsights Reporting\n\nRadar Blog\nBuy Courses\nClose\nSearch\n\n\n\nPlans\nSign In\nTry Now\nO’Reilly Platform\n\n\n\n\n  \n\n\nToggle dark mode\n\n\n  \n\nAI & ML\nBusiness\nData\nInnovation\nResearch\nSecurityO’Reilly LiveJun11Building AI Agents with Model Context Protocol (MCP)Jun17Building GenAI Apps with ChatGPT API and LangChainJun25GitHub Copilot for Software Engineers\nSee allTry the O

In [16]:
type(splitted_documents)

list

In [ ]:
#Initialize embedding model
from langchain_openai import OpenAIEmbeddings
embedding_model = OpenAIEmbeddings()

In [18]:
#Initialize vector store
from langchain_community.vectorstores import FAISS
vectorstoredb = FAISS.from_documents(splitted_documents, embedding_model)

In [19]:
vectorstoredb

In [20]:
query = "What is the main topic of the article?"
result = vectorstoredb.similarity_search(query, k=1)

In [22]:
print(result[0].page_content)

SubscribePlease read our privacy policy.Thank you for subscribing to the O’Reilly Radar Trends to Watch newsletter.   
Radar
> Topics
> AI & MLMCP: What It Is and Why It Matters—Part 3


In [ ]:
## Retrieval chain, Document Chain.(Adds up context to the prompt)
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>


"""
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x112d737c0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x112b5aec0>, root_client=<openai.OpenAI object at 0x112d73dc0>, root_async_client=<openai.AsyncOpenAI object at 0x112b59cf0>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_docume

In [24]:
from langchain_core.documents import Document
document_chain.invoke(
    {
        "input": query,
        "context":[Document(page_content="LangSmith has two usage limits: total traces and extended traces. These correspond to the two metrics we've been tracking on our usage graph. ")]
    }
)

'LangSmith has two usage limits: total traces and extended traces. These limits correspond to the two metrics that have been tracked on the usage graph.'

In [25]:
#Lets use the retriever directly to make sure it acts as an interface
retriever = vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever,document_chain )

In [26]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x1145c7790>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
            | ChatOpe

In [28]:
response_from_retrieval_chain = retrieval_chain.invoke(
    {
        "input": query
    }
)
response_from_retrieval_chain['answer']

'What is the title of the article mentioned in the provided context?'

In [29]:
response_from_retrieval_chain['context']

[Document(id='2fa95c27-096a-435a-88eb-21b57ac0eec8', metadata={'source': 'https://www.oreilly.com/radar/mcp-what-it-is-and-why-it-matters-part-3/', 'title': '\nMCP: What It Is and Why It Matters—Part 3 – O’Reilly', 'description': '', 'language': 'en-US'}, page_content='SubscribePlease read our privacy policy.Thank you for subscribing to the O’Reilly Radar Trends to Watch newsletter.   \nRadar\n> Topics\n> AI & MLMCP: What It Is and Why It Matters—Part 3'),
 Document(id='ab46183d-24e2-40b2-94af-fa3c0c9768bc', metadata={'source': 'https://www.oreilly.com/radar/mcp-what-it-is-and-why-it-matters-part-3/', 'title': '\nMCP: What It Is and Why It Matters—Part 3 – O’Reilly', 'description': '', 'language': 'en-US'}, page_content='By Addy Osmani June 5, 2025 • 17 minute read\nShare'),
 Document(id='7bf7d831-9787-4416-bfe0-e375d8fb7210', metadata={'source': 'https://www.oreilly.com/radar/mcp-what-it-is-and-why-it-matters-part-3/', 'title': '\nMCP: What It Is and Why It Matters—Part 3 – O’Reilly',